In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
import cv2
from tensorflow.keras import layers,models
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

In [2]:
data=np.zeros((5631,128,128,3))
label=np.zeros((5631))
count=0

for file in os.listdir("data/green_area"):
    img = plt.imread('data/green_area/'+file)
    img_t=cv2.resize(img,dsize=(128,128))
    data[count]=img_t
    label[count]=0
    count+=1

for file in os.listdir("data/water"):
    img = plt.imread('data/water/'+file)
    img_t=cv2.resize(img,dsize=(128,128))
    data[count]=img_t
    label[count]=1
    count+=1

for file in os.listdir("data/desert"):
    img = plt.imread('data/desert/'+file)
    img_t=cv2.resize(img,dsize=(128,128))
    data[count]=img_t
    label[count]=2
    count+=1

for file in os.listdir("data/cloudy"):
    img = plt.imread('data/cloudy/'+file)[:,:,:3]
    img_t=cv2.resize(img,dsize=(128,128))
    data[count]=img_t
    label[count]=3
    count+=1

In [3]:
y=np.zeros((label.shape[0],4),dtype='i')
for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        if(label[i]==j):
            y[i,j]=1

In [4]:
data=data/255.0

In [5]:
idx=np.random.permutation(data.shape[0])

In [6]:
train=np.copy(data)
test=np.copy(y)
train,test=train[idx],test[idx]

In [7]:
x_train=train[:4400]
y_train=test[:4400]
x_valid=train[4400:]
y_valid=test[4400:]

In [8]:
input_shape=(128,128,3)

In [9]:
cnn=models.Sequential([
    
    layers.Conv2D(filters=16,kernel_size=(3,3),activation='relu',input_shape=input_shape),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=input_shape),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(4,activation='softmax')]
)

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
cnn.compile(optimizer=tf.keras.optimizers.Adam(0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
callbacks=[
    ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=4),
    EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=False)
]

In [14]:
epochs=60

In [13]:
cnn.fit(x_train,y_train,
        epochs=epochs,
        validation_data=(x_valid,y_valid),
        callbacks=callbacks
       )

Epoch 1/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 50s 246ms/step - accuracy: 0.8479 - loss: 0.4346 - val_accuracy: 0.2900 - val_loss: 1.8243 - learning_rate: 1.0000e-04
Epoch 2/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 233ms/step - accuracy: 0.9496 - loss: 0.1406 - val_accuracy: 0.3014 - val_loss: 2.2697 - learning_rate: 1.0000e-04
Epoch 3/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 232ms/step - accuracy: 0.9564 - loss: 0.1350 - val_accuracy: 0.5361 - val_loss: 2.5612 - learning_rate: 1.0000e-04
Epoch 4/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 34s 243ms/step - accuracy: 0.9623 - loss: 0.1148 - val_accuracy: 0.7027 - val_loss: 0.8549 - learning_rate: 1.0000e-04
Epoch 5/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 31s 224ms/step - accuracy: 0.9611 - loss: 0.1016 - val_accuracy: 0.8652 - val_loss: 0.3276 - learning_rate: 1.0000e-04
Epoch 6/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 229ms/step - accuracy: 0.9754 - loss: 0.0799 - val_accuracy: 0.8562 - val_loss: 0.5625 - learning_rate: 1.0000e-04
Epoch 7/40
138/138 ━━━━━━━━━━━━━━━━━━━━ 32s 22

In [15]:
cnn.evaluate(x_valid,y_valid)

39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.9925 - loss: 0.0199


[0.016779309138655663, 0.9935012459754944]

In [16]:
y_predict=cnn.predict(x_valid)

39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step


In [17]:
def plot(n):
    fig=plt.figure(figsize=(16,16))
    c=4
    r=8
    ax=[]
    for i in range(1,c*r+1):
        j=(32*n)+i
        ax.append(fig.add_subplot(8,4,i))
        if(y_predict[j].argmax()==0):
            if(y_valid[j].argmax()==0):
                ax[-1].yaxis.label.set_color('green')
                plt.ylabel("forest")
            else:
                ax[-1].yaxis.label.set_color('red')
                plt.ylabel("forest")
        elif(y_predict[j].argmax()==1):
            if(y_valid[j].argmax()==1):
                ax[-1].yaxis.label.set_color('green')
                plt.ylabel("lake")
            else:
                ax[-1].yaxis.label.set_color('red')
                plt.ylabel("lake")
        elif(y_predict[j].argmax()==2):
            if(y_valid[j].argmax()==2):
                ax[-1].yaxis.label.set_color('green')
                plt.ylabel("desert")
            else:
                ax[-1].yaxis.label.set_color('red')
                plt.ylabel("desert")
        elif(y_predict[j].argmax()==3):
            if(y_valid[j].argmax()==3):
                ax[-1].yaxis.label.set_color('green')
                plt.ylabel("cloud")
            else:
                ax[-1].yaxis.label.set_color('red')
                plt.ylabel("cloud")
        plt.imshow(x_valid[j])
    plt.show()
    

In [ ]:
for i in range(38):
    plot(i)